In [1]:
# The following section only needs to be executed when running off of google drive

# from google.colab import drive
# drive.mount('/content/drive')

# This needs to be run only once at the beginning to access the models
#-------------------------------------------------------------------------------
!pip install --upgrade git+https://github.com/roschkoenig/SodMod.git


  Cloning https://github.com/roschkoenig/SodMod.git to /tmp/pip-req-build-ju28gaq5
  Running command git clone -q https://github.com/roschkoenig/SodMod.git /tmp/pip-req-build-ju28gaq5
  Created wheel for sodmod: filename=sodmod-0.1-cp36-none-any.whl size=9843 sha256=25b26ac3c742fc7c45934482ca8857f32100b3a2a3a70b213cfb527eeb145501
  Stored in directory: /tmp/pip-ephem-wheel-cache-06x4j7f3/wheels/07/ed/a1/0bc1410b394766c57f600844bcb0d8e79e024d8e5633a74295
Successfully built sodmod
  Found existing installation: sodmod 0.1
    Uninstalling sodmod-0.1:
      Successfully uninstalled sodmod-0.1


In [12]:
import matplotlib.pyplot as plt
import numpy as np
from importlib import reload
from scipy.integrate import odeint
# from torch import multiprocessing as mp
import multiprocessing as mp
from itertools import repeat

import sodmod as sm
from sodmod import params as pr
from sodmod import cells as cl
from sodmod import synapse as sy

In [25]:
reload(sm)
reload(pr)
reload(cl)
reload(sy)

# Simulations
#-------------------------------------------------------------------------------
V0         = -80
steps      = 150

def net_initialise(A, celldefs, V0=-80):
    Y0 = []
    P = []
    k = 0
    
    for c in celldefs:
        p     = pr.params(c[1], typ=c[0], paradigm=c[2])
        y0    = pr.initialise(p,V0,'random')
        
        # Pull out connectivity parameters 
        #---------------------------------------------------------------------------
        stypes = A.keys()
        for s in stypes:
            inconx = A[s][k,:]
            for i in np.where(inconx)[0]: 
                p['snames'].append('r'+s +'_'+ str(k) +'_'+ str(i))
                np.append(y0, 0)
        

        y0[0] = V0
        Y0.append(y0)
        P.append({'par':p, 'ctyp':c[0], 'cond':c[1], 'paradigm':c[2]})
        k = k + 1
    
    return [Y0,P]

def net(Y,t,P): 
    dY = []
    for k in range(len(Y)):
        y           = Y[k]
        cell        = getattr(cl, P[k]['ctyp'])
        s['GABA']   = np.divide([rows[0] for rows in Y], P[k]['inGABA'])
        s['GABA'][s['GABA'] == -np.inf] = np.nan

        dY.append(cell(y,t,P[k]['par'],s))

    return dY 

# Define the particular network structure you want
#-------------------------------------------------------------------------------
celldefs = [['PY', 'WT37', 'constant'],['IN', 'WT37', 'constant']]
A = {}
A['GABA'] = np.zeros((2,2))
A['GABA'][0,1] = 1
[Y0,P] = net_initialise(A, celldefs)


In [27]:
P[1]['par']['snames']

['Vm', 'm_K', 'm_Na', 'h_Na', 'm_NaP']

In [ ]:
# Solve ODE system
#===============================================================================
# Initial conditions
#-------------------------------------------------------------------------------
np.random.seed(1000)

#===============================================================================
# Simulation executor
#===============================================================================
def runsim(i_scl, conds, cell, V0, Y0 = None):

  Vy    = {}

  # Run simulation across conditions
  #-------------------------------------------------------------------------------  
  for ci in range(len(conds)):
    par       = pr.params(conds[ci], i_scl, ctyp, paradigm)
    if Y0 == None:  
      y0    = np.random.rand(len(par['snames']))
      y0[0] = V0
    else:           y0 = Y0[conds[ci]][-1,:]
    Vy.update({conds[ci]:odeint(cell, y0, T, args=(par,))})

  return Vy

array([0.     , 0.     , 0.     , 0.     , 0.15158])

In [ ]:
?np.multiply